In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
from sklearn.model_selection import train_test_split
from tensorflow.random import set_seed
set_seed(69)

import missingno as msno
%matplotlib inline

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from fancyimpute import SoftImpute, MatrixFactorization, BiScaler
# from fancyimpute import IterativeSVD, NuclearNormMinimization, SimilarityWeightedAveraging
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# # By default, a module has some hidden variables defined
# print({k: v for k, v in globals().items() if not k.startswith("__")})

In [3]:
''' 
data dim = [8, 8, 13, 8, 4, 11, 6]
train size = [765, 574, 371, 6064, 7077, 1187, 231]
test size = [309, 230, 151, 2457, 2870, 479, 92]
train / test ~= 2.5
'''
for i in range(1,8,1):
    temp0_df = pd.read_csv(f"data{i}/train.csv", header=None)
    globals()[f'train{i}'] = temp0_df.iloc[:,:-1]
    globals()[f'answer{i}'] = temp0_df.iloc[:,-1]
    temp1_df = pd.read_csv(f"data{i}/test.csv", header=None)
    globals()[f'test{i}'] = temp1_df.iloc[:,:-1]

In [4]:
for i in range(1,8,1):
    count_null = pd.DataFrame(globals()[f'train{i}'].isnull().sum()).T
    display(count_null)

,0,1,2,3,4,5,6,7
0,234,220,218,225,219,239,249,237


,0,1,2,3,4,5,6,7
0,167,161,195,179,151,163,167,173


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,110,124,103,103,105,118,124,122,118,105,104,110,102


,0,1,2,3,4,5,6,7
0,1755,1864,1834,1811,1764,1789,1850,1893


,0,1,2,3
0,2132,2149,2089,2114


,0,1,2,3,4,5,6,7,8,9,10
0,367,395,362,357,351,327,342,367,346,366,321


,0,1,2,3,4,5
0,68,62,74,63,78,68


In [5]:
for i in range(1,8,1):
    count_unique = pd.DataFrame(globals()[f'train{i}'])
    display(count_unique.agg(['nunique']))

,0,1,2,3,4,5,6,7
nunique,216,141,104,151,89,204,224,14


,0,1,2,3,4,5,6,7
nunique,12,12,7,4,2,4,4,6


,0,1,2,3,4,5,6,7,8,9,10,11,12
nunique,259,23,63,2,74,239,204,226,9,51,40,189,252


,0,1,2,3,4,5,6,7
nunique,4013,3960,3962,3982,4010,3999,3958,3900


,0,1,2,3
nunique,2302,611,2053,3241


,0,1,2,3,4,5,6,7,8,9,10
nunique,90,127,78,76,121,52,131,98,82,81,55


,0,1,2,3,4,5
nunique,5,10,8,17,10,14


In [7]:
for i in range(1,8,1):
    count_null = pd.DataFrame(globals()[f'test{i}'].isnull().sum()).T
    display(count_null)

,0,1,2,3,4,5,6,7
0,100,91,93,90,86,92,96,95


,0,1,2,3,4,5,6,7
0,68,65,75,58,69,70,88,80


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,52,37,50,55,43,42,56,50,42,39,42,40,37


,0,1,2,3,4,5,6,7
0,735,731,707,695,764,744,740,732


,0,1,2,3
0,829,896,847,876


,0,1,2,3,4,5,6,7,8,9,10
0,153,124,144,143,163,141,126,145,165,139,168


,0,1,2,3,4,5
0,27,34,25,29,27,29


In [8]:
for i in range(1,8,1):
    count_unique = pd.DataFrame(globals()[f'test{i}'])
    display(count_unique.agg(['nunique']))

,0,1,2,3,4,5,6,7
nunique,129,77,53,94,65,123,135,11


,0,1,2,3,4,5,6,7
nunique,12,12,7,4,2,4,4,6


,0,1,2,3,4,5,6,7,8,9,10,11,12
nunique,88,18,44,2,51,93,75,86,9,42,30,77,99


,0,1,2,3,4,5,6,7
nunique,1453,1462,1471,1474,1415,1460,1448,1463


,0,1,2,3
nunique,1313,531,1171,1481


,0,1,2,3,4,5,6,7,8,9,10
nunique,72,91,69,43,85,44,101,80,69,65,48


,0,1,2,3,4,5
nunique,5,9,8,14,10,14


In [9]:
def make_upload(imputer, model = LGBMRegressor(random_state=42), scaler = None, status = False, test_size = 0.15, seed = 42):
    tp_mean = 0.0
    for i in range(1,8,1):
        np_train, np_answer = np.array(globals()[f'train{i}']), np.array(globals()[f'answer{i}'])
        X_train, X_valid, y_train, y_valid = train_test_split(np_train, np_answer, test_size=test_size, random_state=seed)
        if scaler is True:
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.fit_transform(X_valid)

        ft_train = imputer.fit_transform(X_train)
        if "fancyimpute" in imputer.__module__:
            ft_valid = imputer.fit_transform(X_valid)
        else: ft_valid = imputer.transform(X_valid)

        model.fit(ft_train, y_train)
        output = model.predict(ft_valid)
        mae = mean_absolute_error(y_valid, output)
        tp_mean += mae
        print(mae, ft_valid.shape, end=' ')

        if status is True:
            X_test = np.array(globals()[f'test{i}'])
            if scaler is True: scaler.fit_transform(X_test)
            if "fancyimpute" in imputer.__module__:
                ft_test = imputer.fit_transform(X_test)
            else: ft_test = imputer.transform(X_test)
            print(ft_test.shape, end='')
            predict = model.predict(ft_test)
            upload_array = np.concatenate((ft_test, predict[:, np.newaxis]), axis=1)
            upload = pd.DataFrame(upload_array)
            upload.to_csv(f'upload/{str(imputer)[:3]}_{str(model)[:4]}_upload{i}.csv', header=None, index=None, sep=',', mode='w')
        print()
    print("TP_mean in valid set = {}".format(tp_mean))

In [9]:
make_upload(IterativeImputer(max_iter=100, tol=1e-2, initial_strategy='constant', random_state=42), status=True) # tol=5e-4

7.9900834718780605 (115, 8) (309, 8)
2.0631783697673503 (87, 8) (230, 8)
2.9089243453393387 (56, 13) (151, 13)
0.15943843320117526 (910, 8) (2457, 8)
4.2860188572932625 (1062, 4) (2870, 4)
0.45959907088924007 (179, 11) (479, 11)
9.865238485356905 (35, 6) (92, 6)
TP_mean in valid set = 27.732481033725332


In [10]:
make_upload(KNNImputer(weights="distance"), LGBMRegressor(random_state=42), status = False)

8.048162326806576 (115, 8) 
2.171453452118326 (87, 8) 
3.101804473338446 (56, 13) 
0.17042330951206006 (910, 8) 
5.065000907165189 (1062, 4) 
0.5048186419084653 (179, 11) 
9.648983123074908 (35, 6) 
TP_mean in valid set = 28.710646233923967


In [11]:
make_upload(SoftImpute(None, 1e-5, 10000, max_rank=10, init_fill_method='zero', verbose=False, seed=42), \
    scaler=BiScaler(scale_rows=False, scale_columns=False, verbose=False))

9.007934660291552 (115, 8) 
2.563748448025553 (87, 8) 
3.794182009673096 (56, 13) 
0.1622370950845623 (910, 8) 
4.619672741376433 (1062, 4) 
0.4897435537636039 (179, 11) 
9.299635389313757 (35, 6) 
TP_mean in valid set = 29.937153897528553


In [12]:
make_upload(MatrixFactorization(learning_rate=1e-4, patience=20, verbose=False), status=False)

9.079174667240352 (115, 8) 
2.859027993373506 (87, 8) 
3.837503417263275 (56, 13) 
0.16192137955131766 (910, 8) 
4.690860565054495 (1062, 4) 
0.5005922014474237 (179, 11) 
9.329345966251289 (35, 6) 
TP_mean in valid set = 30.458426190181658


In [23]:
# Convert to .txt file
for i in range(1,8,1):
    PATH = "GRAPE/uci/raw_data"
    os.makedirs(f'{PATH}/data{i}/data', exist_ok=True)
    data = globals()[f'train{i}'].copy()
    data.fillna(value = 0, inplace=True)
    data.to_csv(f'{PATH}/data{i}/data/data.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
'''
opt : adam, sgd, rmsprop, adagrad; opt_scheduler : step, cos
'''
!python GRAPE/train_mdi.py --epochs 200 --opt_scheduler cos --opt_decay_step 50 --opt_decay_rate 0.9 --weight_decay 1e-5 \
 --valid 0.1 --save_model --save_prediction uci --data data1

In [ ]:
!python GRAPE/train_y.py --epochs 200 --opt_scheduler cos --opt_decay_step 50 --opt_decay_rate 0.9 --weight_decay 1e-5 \
--valid 0.1 uci --data data1

In [27]:
# 檢視 Feature imputation 訓練數據
import pickle
fr = open('uci/test/data1/0/result.pkl', 'rb')
df = pickle.load(fr)

# print('Items of result.pkl: ', df.keys())
# print('\nargs: ', df['args'])
print('\noutputs: ', df['outputs'])
# print('\ncurves: ', df['curves'])
# print('\nlr: ', df['lr'])


outputs:  {'best_valid_rmse_pred_test': array([0.23472586, 0.3429135 , 0.49465045, ..., 0.48028755, 0.48039582,
       0.4804244 ], dtype=float32), 'best_valid_l1_pred_test': array([0.08135975, 0.3113982 , 0.68225396, ..., 0.58054477, 0.6309263 ,
       0.6309227 ], dtype=float32), 'final_pred_train': array([0.17951287, 0.0942234 , 0.3507589 , ..., 0.28867045, 0.20371333,
       0.6800189 ], dtype=float32), 'label_train': array([0.        , 0.        , 0.        , ..., 0.39130434, 0.        ,
       0.7671771 ], dtype=float32), 'final_pred_test': array([0.07680242, 0.29278088, 0.6853763 , ..., 0.5381242 , 0.627933  ,
       0.6432092 ], dtype=float32), 'label_test': array([0.       , 0.       , 0.       , ..., 0.7877193, 0.8798246,
       0.7650443], dtype=float32)}


In [36]:
print(df['outputs'].keys())
for key in df['outputs'].keys():
    print("{} ".format(df['outputs'][key].shape), end='')

dict_keys(['best_valid_rmse_pred_test', 'best_valid_l1_pred_test', 'final_pred_train', 'label_train', 'final_pred_test', 'label_test'])
(1547,) (1547,) (3396,) (3396,) (1547,) (1547,) 

In [ ]:
# 檢視 Label prediction 訓練數據
fr = open('uci/test/data1/y0/result.pkl', 'rb')
df = pickle.load(fr)

# print('Items of result.pkl: ', df.keys())
# print('\nargs: ', df['args'])
print('\noutputs: ', df['outputs'])
# print('\ncurves: ', df['curves'])
# print('\nlr: ', df['lr'])

In [38]:
print(df['outputs'].keys())
for key in df['outputs'].keys():
    print("{} ".format(df['outputs'][key].shape), end='')

dict_keys(['pred_train', 'label_train', 'pred_test', 'label_test'])
(454,) (454,) (242,) (242,) 